In [ ]:
!pip install -U adapter-transformers
!pip install datasets

     |████████████████████████████████| 2.5 MB 15.3 MB/s 
     |████████████████████████████████| 3.3 MB 64.8 MB/s 
     |████████████████████████████████| 895 kB 75.8 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 264 kB 15.1 MB/s 
     |████████████████████████████████| 243 kB 27.6 MB/s 
     |████████████████████████████████| 118 kB 24.7 MB/s 
     |████████████████████████████████| 76 kB 5.1 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [ ]:
import pandas as pd
import json

def jsonl2txt(path):
    df = pd.DataFrame()
    with open(path) as f:
        for line in f:
            df = df.append(json.loads(line), ignore_index=True)
    df.text.to_csv(path.replace("jsonl", "txt"), header=False, index=False)
jsonl2txt('../data/ag_train.jsonl')
jsonl2txt('../data/ag_dev.jsonl')
jsonl2txt('../data/ag_test.jsonl')

In [ ]:
jsonl2txt('../data/hyperpartisan_news_dev.jsonl')
jsonl2txt('../data/hyperpartisan_news_train.jsonl')
jsonl2txt('../data/hyperpartisan_news_test.jsonl')

In [ ]:
jsonl2txt('../data/rct-sample_dev.jsonl')
jsonl2txt('../data/rct-sample_train.jsonl')
jsonl2txt('../data/rct-sample_test.jsonl')

In [ ]:
jsonl2txt('../data/chemprot_dev.jsonl')
jsonl2txt('../data/chemprot_test.jsonl')
jsonl2txt('../data/chemprot_train.jsonl')

In [ ]:
jsonl2txt('../data/sciie_dev.jsonl')
jsonl2txt('../data/sciie_test.jsonl')
jsonl2txt('../data/sciie_train.jsonl')




**V2-1: Creating new adapter using hyperpartisan news dataset**

In [ ]:
!python3 run_mlm.py \
--train_file data/hyperpartisan_news_train.txt \
--line_by_line \
--validation_file data/hyperpartisan_news_dev.txt \
--model_name_or_path roberta-base \
--mlm_probability 0.15 \
--do_train \
--do_eval \
--output_dir results/adapters/hyperpartisan_news \
--train_adapter \
--num_train_epochs 100 \
--learning_rate 1e-4 \
--logging_steps 50 \
--per_gpu_train_batch_size 8 \
--per_gpu_eval_batch_size 8 \
--gradient_accumulation_steps 8  \

2021-08-02 07:30:51.133206: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/02/2021 07:30:52 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/02/2021 07:30:52 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=50,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=8,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=0.0001,
lengt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/dl_final_project/gatech_deep_final"
!ls

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/dl_final_project/gatech_deep_final"
!ls

/content/drive/MyDrive/dl_final_project/gatech_deep_final
'~'	       run_mlm.py
 data	       run_multiple_choice_downstream_task.py
 experiments   run_multiple_choice.py
 __pycache__   tapos_test.ipynb
 README.md     tapos-training
 results       utils_multiple_choice.py


V2-2: Use new adapter hyperpartisan on hyperpartisan dataset

In [ ]:
!python3 run_multiple_choice.py \
--do_train \
--do_eval \
--data_dir data/hyperpartisan_news_ \
--max_seq_length 512 \
--per_device_train_batch_size 8 \
--gradient_accumulation_steps 1 \
--learning_rate 2e-5 \
--num_train_epochs 30 \
--output_dir results/hyperpartisan_news-new-adapter/ \
--task_name mlm \
--do_predict \
--model_name_or_path roberta-base \
--adapter_1 results/adapters/hyperpartisan_news/mlm \
--load_best_model_at_end \

2021-08-02 09:07:05.115631: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/02/2021 09:07:06 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassif

In [ ]:
!python3 run_multiple_choice.py \
--do_train \
--do_eval \
--data_dir data/hyperpartisan_news_ \
--max_seq_length 512 \
--per_device_train_batch_size 4 \
--gradient_accumulation_steps 1 \
--learning_rate 2e-5 \
--num_train_epochs 30 \
--output_dir results/hyperpartisan_news-new-adapter3/ \
--task_name mlm \
--do_predict \
--model_name_or_path roberta-base \
--adapter_1 results/adapters/hyperpartisan_news/mlm \
--load_best_model_at_end \

2021-08-02 18:01:59.173418: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/02/2021 18:02:00 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
08/02/2021 18:02:03 - INFO - filelock - Lock 139966987825616 acquired on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
Downloading: 100% 481/481 [00:00<00:00, 518kB/s]
08/02/2021 18:02:03 - INFO - filelock - Lock 139966987825616 released on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
08/02/2021 18:02:04 - INFO - filelock - Lock 139966987740944 acquired on /root/.cache/huggingface/transformers/d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fd

V2-3: Tuning new adapter hyperpartisan on hyperpartisan dataset

In [ ]:
!python3 run_multiple_choice.py \
--do_train \
--do_eval \
--data_dir data/hyperpartisan_news_ \
--max_seq_length 512 \
--per_device_train_batch_size 4 \
--gradient_accumulation_steps 1 \
--learning_rate 2e-5 \
--num_train_epochs 45 \
--output_dir results/hyperpartisan_news-new-adapter4/ \
--task_name mlm \
--do_predict \
--model_name_or_path roberta-base \
--adapter_1 results/adapters/hyperpartisan_news/mlm \
--load_best_model_at_end \

2021-08-02 18:38:23.538676: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/02/2021 18:38:25 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassif

In [ ]:
!python3 run_multiple_choice.py \
--do_train \
--do_eval \
--data_dir data/hyperpartisan_news_ \
--max_seq_length 512 \
--per_device_train_batch_size 12 \
--gradient_accumulation_steps 1 \
--learning_rate 2e-5 \
--num_train_epochs 45 \
--output_dir results/hyperpartisan_news-new-adapter5/ \
--task_name mlm \
--do_predict \
--model_name_or_path roberta-base \
--adapter_1 results/adapters/hyperpartisan_news/mlm \
--load_best_model_at_end \

2021-08-02 19:28:59.472598: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/02/2021 19:29:00 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassif

In [ ]:
!python3 run_multiple_choice.py \
--do_train \
--do_eval \
--data_dir data/hyperpartisan_news_ \
--max_seq_length 512 \
--per_device_train_batch_size 12 \
--gradient_accumulation_steps 1 \
--learning_rate 2e-5 \
--num_train_epochs 20 \
--output_dir results/hyperpartisan_news-new-adapter6/ \
--task_name mlm \
--do_predict \
--model_name_or_path roberta-base \
--adapter_1 results/adapters/hyperpartisan_news/mlm \
--load_best_model_at_end \

2021-08-02 20:10:19.745583: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/02/2021 20:10:21 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassif

In [ ]:
!python3 run_multiple_choice.py \
--do_train \
--do_eval \
--data_dir data/hyperpartisan_news_ \
--max_seq_length 512 \
--per_device_train_batch_size 4 \
--gradient_accumulation_steps 1 \
--learning_rate 2e-5 \
--num_train_epochs 20 \
--output_dir results/hyperpartisan_news-new-adapter6/ \
--task_name mlm \
--do_predict \
--model_name_or_path roberta-base \
--adapter_1 results/adapters/hyperpartisan_news/mlm \
--load_best_model_at_end \

2021-08-02 20:31:59.132446: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/02/2021 20:32:00 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassif

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/dl_final_project/gatech_deep_final"
!ls

Mounted at /content/drive
/content/drive/MyDrive/dl_final_project/gatech_deep_final
'~'	       run_mlm.py
 data	       run_multiple_choice_downstream_task.py
 experiments   run_multiple_choice.py
 __pycache__   tapos_test.ipynb
 README.md     tapos-training
 results       utils_multiple_choice.py


V2-Ag: New adapter hyperpartisan on ag news dataset

In [ ]:
!python3 run_multiple_choice.py \
--do_train \
--do_eval \
--data_dir data/ag_ \
--max_seq_length 512 \
--per_device_train_batch_size 8 \
--gradient_accumulation_steps 1 \
--learning_rate 2e-5 \
--num_train_epochs 2 \
--output_dir results/hyperpartisan_news-new-adapter9/ \
--task_name mlm \
--do_predict \
--model_name_or_path roberta-base \
--adapter_1 results/adapters/hyperpartisan_news/mlm \
--load_best_model_at_end \
--overwrite_output_dir \

2021-08-02 21:45:52.630385: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/02/2021 21:45:54 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
08/02/2021 21:45:57 - INFO - filelock - Lock 140064468737872 acquired on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
Downloading: 100% 481/481 [00:00<00:00, 444kB/s]
08/02/2021 21:45:58 - INFO - filelock - Lock 140064468737872 released on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
08/02/2021 21:45:59 - INFO - filelock - Lock 140064616791248 acquired on /root/.cache/huggingface/transformers/d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/dl_final_project/gatech_deep_final"
!ls

/content/drive/MyDrive/dl_final_project/gatech_deep_final
'~'	       run_mlm.py
 data	       run_multiple_choice_downstream_task.py
 experiments   run_multiple_choice.py
 __pycache__   tapos_test.ipynb
 README.md     tapos-training
 results       utils_multiple_choice.py


V2-4: New adapter hyperpartisan on imdb dataset

In [ ]:
!python3 run_multiple_choice.py \
--do_train \
--do_eval \
--data_dir data/imdb_ \
--max_seq_length 512 \
--per_device_train_batch_size 8 \
--gradient_accumulation_steps 1 \
--learning_rate 2e-5 \
--num_train_epochs 2 \
--output_dir results/hyperpartisan_news-new-adapter10/ \
--task_name mlm \
--do_predict \
--model_name_or_path roberta-base \
--adapter_1 results/adapters/hyperpartisan_news/mlm \
--load_best_model_at_end \
--overwrite_output_dir \

2021-08-03 04:39:47.182260: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/03/2021 04:39:49 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
08/03/2021 04:39:53 - INFO - filelock - Lock 140679781742288 acquired on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
Downloading: 100% 481/481 [00:00<00:00, 455kB/s]
08/03/2021 04:39:53 - INFO - filelock - Lock 140679781742288 released on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
08/03/2021 04:39:54 - INFO - filelock - Lock 140679781741072 acquired on /root/.cache/huggingface/transformers/d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fd

**V2**-5: New adapter hyperpartisan on amazon dataset

In [ ]:
 !python3 run_multiple_choice.py \
--do_train \
--do_eval \
--data_dir data/amazon_ \
--max_seq_length 512 \
--per_device_train_batch_size 8 \
--gradient_accumulation_steps 1 \
--learning_rate 2e-5 \
--num_train_epochs 2 \
--output_dir results/hyperpartisan_news-new-adapter11/ \
--task_name mlm \
--do_predict \
--model_name_or_path roberta-base \
--adapter_1 results/adapters/hyperpartisan_news/mlm \
--load_best_model_at_end \

2021-08-03 05:09:59.831977: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/03/2021 05:10:01 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassif

In [ ]:
!python3 run_multiple_choice.py \
--do_train \
--do_eval \
--data_dir data/rct-sample_ \
--max_seq_length 512 \
--per_device_train_batch_size 8 \
--gradient_accumulation_steps 1 \
--learning_rate 2e-5 \
--num_train_epochs 30 \
--output_dir results/hyperpartisan_news-new-adapter12/ \
--task_name mlm \
--do_predict \
--model_name_or_path roberta-base \
--adapter_1 results/adapters/hyperpartisan_news/mlm \
--load_best_model_at_end \
--overwrite_output_dir \

2021-08-03 07:33:19.868497: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/03/2021 07:33:21 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassif

V2-5: New adapter hyperpartisan on rct_sample dataset

In [ ]:
!python3 run_multiple_choice.py \
--do_train \
--do_eval \
--data_dir data/rct-sample_ \
--max_seq_length 512 \
--per_device_train_batch_size 8 \
--gradient_accumulation_steps 1 \
--learning_rate 2e-5 \
--num_train_epochs 30 \
--output_dir results/hyperpartisan_news-new-adapter12/ \
--task_name mlm \
--do_predict \
--model_name_or_path roberta-base \
--adapter_1 results/adapters/hyperpartisan_news/mlm \
--load_best_model_at_end \

2021-08-03 07:48:19.285803: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/03/2021 07:48:20 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassif

V2-6: New adapter hyperpartisan on citation-intent dataset

In [ ]:
!python3 run_multiple_choice.py \
--do_train \
--do_eval \
--data_dir data/citation-intent_ \
--max_seq_length 512 \
--per_device_train_batch_size 8 \
--gradient_accumulation_steps 1 \
--learning_rate 2e-5 \
--num_train_epochs 30 \
--output_dir results/hyperpartisan_news-new-adapter13/ \
--task_name mlm \
--do_predict \
--model_name_or_path roberta-base \
--adapter_1 results/adapters/hyperpartisan_news/mlm \
--load_best_model_at_end \

2021-08-03 08:33:20.841876: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/03/2021 08:33:22 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassif

Best Performance with Macro F1

In [ ]:
!python3 run_multiple_choice.py \
--do_train \
--do_eval \
--data_dir data/hyperpartisan_news_ \
--max_seq_length 512 \
--per_device_train_batch_size 12 \
--gradient_accumulation_steps 1 \
--learning_rate 2e-5 \
--num_train_epochs 45 \
--output_dir results/hyperpartisan_news_adapterhub_adapters10/ \
--task_name hb3 \
--do_predict \
--model_name_or_path roberta-base \
--adapter_1 results/adapters/ag/mlm \
--load_best_model_at_end \
--seed 5

2021-08-03 09:55:13.995554: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/03/2021 09:55:15 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassif

only new two adapter

In [ ]:
!python3 run_multiple_choice.py \
--do_train \
--do_eval \
--data_dir data/hyperpartisan_news_ \
--max_seq_length 512 \
--per_device_train_batch_size 12 \
--gradient_accumulation_steps 1 \
--learning_rate 2e-5 \
--num_train_epochs 45 \
--output_dir results/hyperpartisan_news_adapterhub_adapters11/ \
--task_name hb3 \
--do_predict \
--model_name_or_path roberta-base \
--adapter_1 results/adapters/ag/mlm \
--load_best_model_at_end \
--seed 5

2021-08-03 10:17:11.513196: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/03/2021 10:17:13 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassif

In [ ]:
!python3 run_multiple_choice.py \
--do_train \
--do_eval \
--data_dir data/hyperpartisan_news_ \
--max_seq_length 512 \
--per_device_train_batch_size 12 \
--gradient_accumulation_steps 1 \
--learning_rate 2e-5 \
--num_train_epochs 45 \
--output_dir results/hyperpartisan_news_adapterhub_adapters12/ \
--task_name hb3 \
--do_predict \
--model_name_or_path roberta-base \
--adapter_1 results/adapters/ag/mlm \
--load_best_model_at_end \
--seed 5

2021-08-03 10:30:34.303826: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/03/2021 10:30:35 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassif

all adapters

In [ ]:
!python3 run_multiple_choice.py \
--do_train \
--do_eval \
--data_dir data/ag_ \
--max_seq_length 128 \
--per_device_train_batch_size 12 \
--gradient_accumulation_steps 1 \
--learning_rate 2e-5 \
--num_train_epochs 10 \
--output_dir results/hyperpartisan_news_adapterhub_adapters15/ \
--task_name hb3 \
--do_predict \
--model_name_or_path roberta-base \
--adapter_1 results/adapters/hyperpartisan_news/mlm \
--load_best_model_at_end \
--seed 5

2021-08-03 17:25:22.528849: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/03/2021 17:25:24 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassif